In [92]:
%load_ext autoreload
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.insert(0,os.path.dirname(currentdir))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [93]:
%autoreload 2

import pandas as pd

from tools import DataLoader

%aimport

Modules to reload:
all-except-skipped

Modules to skip:



In [94]:
data=DataLoader()
data.jhu()

In [95]:
data.deaths.sort_values("date", ascending=False)#[data.confirmed.date=="index"]#

,region,iso2,iso3,code3,date,deaths
13983,Zimbabwe,ZW,ZWE,716.0,4/6/20,1
7371,Lebanon,LB,LBN,422.0,4/6/20,19
10031,Paraguay,PY,PRY,600.0,4/6/20,5
7067,Kuwait,KW,KWT,414.0,4/6/20,1
9955,Papua New Guinea,PG,PNG,598.0,4/6/20,0
...,...,...,...,...,...,...
9044,Nepal,NP,NPL,524.0,1/22/20,0
8968,Namibia,NaN,NAM,516.0,1/22/20,0
8892,Mozambique,MZ,MOZ,508.0,1/22/20,0
8816,Morocco,MA,MAR,504.0,1/22/20,0


In [96]:
data.ecdc()
data.ecdc_raw.tail(3)

,date,cases,deaths,country,geoId,iso_alpha,population
1712,2020-04-04,1,0,Cayman_Islands,KY,CYM,64174.0
5013,2020-04-04,0,0,Lebanon,LB,LBN,6848925.0
3,2020-04-04,0,0,Afghanistan,AF,AFG,37172386.0


In [97]:
data.geonames()
data.country_info.head(3)

,iso_alpha,iso_num,geoId,Country,Capital,Area(in sq km),population,continent,tld,CurrencyCode,CurrencyName,Phone,Postal Code Format,Postal Code Regex,Languages,geonameid,neighbours,EquivalentFipsCode
0,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU,.ad,EUR,Euro,376,AD###,^(?:AD)*(\d{3})$,ca,3041565,"ES,FR",
1,ARE,784,AE,United Arab Emirates,Abu Dhabi,82880.0,9630959,AS,.ae,AED,Dirham,971,,,"ar-AE,fa,en,hi,ur",290557,"SA,OM",
2,AFG,4,AF,Afghanistan,Kabul,647500.0,37172386,AS,.af,AFN,Afghani,93,,,"fa-AF,ps,uz-AF,tk",1149361,"TM,CN,IR,TJ,PK,UZ",


In [166]:
data.deaths

,region,iso2,iso3,code3,date,deaths
0,Afghanistan,AF,AFG,4.0,1/22/20,0
1,Afghanistan,AF,AFG,4.0,1/23/20,0
2,Afghanistan,AF,AFG,4.0,1/24/20,0
3,Afghanistan,AF,AFG,4.0,1/25/20,0
4,Afghanistan,AF,AFG,4.0,1/26/20,0
...,...,...,...,...,...,...
13979,Zimbabwe,ZW,ZWE,716.0,4/2/20,1
13980,Zimbabwe,ZW,ZWE,716.0,4/3/20,1
13981,Zimbabwe,ZW,ZWE,716.0,4/4/20,1
13982,Zimbabwe,ZW,ZWE,716.0,4/5/20,1


In [169]:
data_norm = pd.merge(data.deaths.sort_values(["region", "date"]), data.country_info[["iso_alpha", "population", "continent"]], left_on="iso3", right_on="iso_alpha", how="inner").drop(columns=["iso3", "iso2", "code3"])
data_norm = pd.merge(data_norm, data.confirmed.sort_values(["region", "date"])[["date","confirmed",  "iso3"]], left_on=["iso_alpha", "date"], right_on=["iso3", "date"], how="inner").drop(columns=["iso3"])
data_norm = pd.merge(data_norm, data.recovered.sort_values(["region", "date"])[["date","recovered",  "iso3"]], left_on=["iso_alpha", "date"], right_on=["iso3", "date"], how="inner").drop(columns=["iso3"])

data_norm[data_norm.iso_alpha=="USA"]

,region,date,deaths,iso_alpha,population,continent,confirmed,recovered
12768,US,1/22/20,0,USA,327167434,NA,1,0
12769,US,1/23/20,0,USA,327167434,NA,1,0
12770,US,1/24/20,0,USA,327167434,NA,2,0
12771,US,1/25/20,0,USA,327167434,NA,2,0
12772,US,1/26/20,0,USA,327167434,NA,5,0
...,...,...,...,...,...,...,...,...
12839,US,4/2/20,5926,USA,327167434,NA,243453,9001
12840,US,4/3/20,7087,USA,327167434,NA,275586,9707
12841,US,4/4/20,8407,USA,327167434,NA,308850,14652
12842,US,4/5/20,9619,USA,327167434,NA,337072,17448


In [154]:
data_norm.loc[:, "confirmed/1M capita"] = (
    data_norm.confirmed / data_norm.population * 1000000
).round(0)
data_norm.loc[:, "deaths/1M capita"] = (
    data_norm.deaths / data_norm.population * 1000000
).round(0)
data_norm.loc[:, "recovered/1M capita"] = (
    data_norm.recovered / data_norm.population * 1000000
).round(0)

data_norm[data_norm.iso_alpha=="USA"].tail()

,region,date,deaths,iso_alpha,population,continent,confirmed,recovered,confirmed/1M capita,deaths/1M capita,recovered/1M capita
12839,US,4/2/20,5926,USA,327167434,NA,243453,9001,744.0,18.0,28.0
12840,US,4/3/20,7087,USA,327167434,NA,275586,9707,842.0,22.0,30.0
12841,US,4/4/20,8407,USA,327167434,NA,308850,14652,944.0,26.0,45.0
12842,US,4/5/20,9619,USA,327167434,NA,337072,17448,1030.0,29.0,53.0
12843,US,4/6/20,10783,USA,327167434,NA,366614,19581,1121.0,33.0,60.0


In [170]:
data_norm

,region,date,deaths,iso_alpha,population,continent,confirmed,recovered
0,Afghanistan,1/22/20,0,AFG,37172386,AS,0,0
1,Afghanistan,1/23/20,0,AFG,37172386,AS,0,0
2,Afghanistan,1/24/20,0,AFG,37172386,AS,0,0
3,Afghanistan,1/25/20,0,AFG,37172386,AS,0,0
4,Afghanistan,1/26/20,0,AFG,37172386,AS,0,0
...,...,...,...,...,...,...,...,...
13751,Zimbabwe,4/2/20,1,ZWE,14439018,AF,9,0
13752,Zimbabwe,4/3/20,1,ZWE,14439018,AF,9,0
13753,Zimbabwe,4/4/20,1,ZWE,14439018,AF,9,0
13754,Zimbabwe,4/5/20,1,ZWE,14439018,AF,9,0


In [171]:
summary_country = data_norm[data_norm.date==data_norm.date.max()]#.groupby("iso_alpha").sum()



aggregation = {
    "cases": "sum",
    "deaths": "sum",
    "country": "max",
    "population": "max",
}
#summary_country = summary_country.agg(aggregation)

summary_country[summary_country.region=="Germany"]

,region,date,deaths,iso_alpha,population,continent,confirmed,recovered
4939,Germany,4/6/20,1810,DEU,82927922,EU,103374,28700


In [176]:
timeseries = data_norm.groupby(["continent", "date"]).sum()

world = timeseries.groupby("date").sum()
world = pd.DataFrame(index=[pd.Series(data="world").repeat(len(world.index)), world.index], data=world.values, columns=world.columns)

timeseries = pd.concat([timeseries, world])
timeseries#.loc["world","confirmed"]
timeseries.loc["NA"].head(50)

,deaths,population,confirmed,recovered
date,,,,
1/22/20,0,578407428,1,0
1/23/20,0,578407428,1,0
1/24/20,0,578407428,2,0
1/25/20,0,578407428,2,0
1/26/20,0,578407428,6,0
1/27/20,0,578407428,6,0
1/28/20,0,578407428,7,0
1/29/20,0,578407428,7,0
1/30/20,0,578407428,7,0


In [122]:
#pd.Series(data="world").repeat(len(world.index))

In [ ]:
summary_country.loc[:, "Cases/Mio. capita"] = (
    summary_country.cases / summary_country.population * 1000000
).round(0)
summary_country.loc[:, "Deaths/Mio. capita"] = (
    summary_country.deaths / summary_country.population * 1000000
).round(0)
summary_country.reset_index(inplace=True)

In [28]:
# check consistency
test = data.confirmed[data.confirmed.iso2=="CN"]
test[test.date=="1/22/20"]

,Country/Region,iso2,iso3,code3,date,confirmed
2736,China,CN,CHN,156.0,1/22/20,548


In [91]:
geonames_country_info = "http://download.geonames.org/export/dump/countryInfo.txt"
country_info = pd.read_csv(geonames_country_info, skiprows=49, delimiter="\t", na_filter=False)
country_info.drop(columns=["#ISO"], inplace=True)
country_info.rename(columns={"fips": "geoId", "ISO3":"iso_alpha", "ISO-Numeric": "iso_num"}, inplace=True)
country_info[country_info.Country=="Canada"].head(3)

,iso_alpha,iso_num,geoId,Country,Capital,Area(in sq km),Population,Continent,tld,CurrencyCode,CurrencyName,Phone,Postal Code Format,Postal Code Regex,Languages,geonameid,neighbours,EquivalentFipsCode
37,CAN,124,CA,Canada,Ottawa,9984670.0,37058856,NA,.ca,CAD,Dollar,1,@#@ #@#,^([ABCEGHJKLMNPRSTVXY]\d[ABCEGHJKLMNPRSTVWXYZ]...,"en-CA,fr-CA,iu",6251999,US,


In [42]:
def create_timeseries(data, lookup, id_vars, var_name, value_name):
    timeseries_d = pd.melt(data, id_vars=id_vars, var_name=var_name, value_name=value_name)
    timeseries = pd.merge(lookup[["iso2", "iso3", "code3", "Country/Region"]].groupby("Country/Region").first(), timeseries_d, on=id_vars, how="right")
    return timeseries

test = create_timeseries(recovered_c, lookup, "Country/Region", "date", "recovered")
test[test.iso2=="CN"]

,Country/Region,iso2,iso3,code3,date,recovered
2664,China,CN,CHN,156.0,1/22/20,28
2665,China,CN,CHN,156.0,1/23/20,30
2666,China,CN,CHN,156.0,1/24/20,36
2667,China,CN,CHN,156.0,1/25/20,39
2668,China,CN,CHN,156.0,1/26/20,49
...,...,...,...,...,...,...
2733,China,CN,CHN,156.0,3/31/20,76206
2734,China,CN,CHN,156.0,4/1/20,76405
2735,China,CN,CHN,156.0,4/2/20,76565
2736,China,CN,CHN,156.0,4/3/20,76760


In [157]:
lookup_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv"
confirmed_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
deaths_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
recovered_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"

In [158]:
lookup = pd.read_csv(lookup_url)
lookup.rename(columns={"Country_Region":"Country/Region"}, inplace=True)
confirmed = pd.read_csv(confirmed_url)
deaths = pd.read_csv(deaths_url)
recovered = pd.read_csv(recovered_url)

In [159]:
confirmed_c = confirmed.groupby("Country/Region").sum().drop(columns=["Lat", "Long"]).reset_index()
deaths_c = deaths.groupby("Country/Region").sum().drop(columns=["Lat", "Long"]).reset_index()
recovered_c = recovered.groupby("Country/Region").sum().drop(columns=["Lat", "Long"]).reset_index()

confirmed_c[confirmed_c["Country/Region"]=="China"]

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20
36,China,548,643,920,1406,2075,2877,5509,6087,8141,...,81999,82122,82198,82279,82361,82432,82511,82543,82602,82665


In [165]:
deaths_c[deaths_c.isna()]

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
recovered_timeseries = pd.melt(recovered_c, id_vars=["Country/Region"], var_name="date", value_name="recovered")
recovered_timeseries[recovered_timeseries["Country/Region"]=="China"]

,Country/Region,date,recovered
36,China,1/22/20,28
217,China,1/23/20,30
398,China,1/24/20,36
579,China,1/25/20,39
760,China,1/26/20,49
...,...,...,...
12525,China,3/31/20,76206
12706,China,4/1/20,76405
12887,China,4/2/20,76565
13068,China,4/3/20,76760


In [18]:
# attache iso2, iso3 and UID to Country/Region
lookup[lookup["Country/Region"]=="China"]

NameError: name 'lookup' is not defined

In [53]:
data.__dict__.keys()

dict_keys(['latest_update', 'countries', 'per_country', 'summary_country', 'ecdc_raw'])

In [36]:
recovered_timeseries_1 = pd.merge(lookup[["iso2", "iso3", "code3", "Country/Region"]], recovered_timeseries, left_on="Country/Region", right_on="Country/Region", how="left").drop(columns=["Country/Region"])
recovered_timeseries_1[recovered_timeseries_1.iso2=="CN"]

,iso2,iso3,code3,Country/Region,date,recovered
16724,CN,CHN,156.0,China,1/22/20,28
16725,CN,CHN,156.0,China,1/23/20,30
16726,CN,CHN,156.0,China,1/24/20,36
16727,CN,CHN,156.0,China,1/25/20,39
16728,CN,CHN,156.0,China,1/26/20,49
...,...,...,...,...,...,...
19087,CN,CHN,156.0,China,3/31/20,76206
19088,CN,CHN,156.0,China,4/1/20,76405
19089,CN,CHN,156.0,China,4/2/20,76565
19090,CN,CHN,156.0,China,4/3/20,76760
